### How to verify the execution of a MBQC pattern ?
Let the following circuit.

In [1]:
import graphix.command
from graphix.random_objects import Circuit
from graphix.states import BasicStates

from veriphix.client import Client, Secrets

/Users/sabdulsa/Codes/veriphix/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
## This circuit, with |0> states as inputs,
# yields deterministic outcomes (BQP error p =0)
circuit = Circuit(2)
circuit.cnot(0,1)
circuit.h(0)
circuit.h(1)
pattern = circuit.transpile().pattern   ## 6 nodes
pattern.standardize()
pattern.print_pattern()

## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))


pattern.standardize()
states = [BasicStates.ZERO for _ in range(2)]

n_good_output = 0

for _i in range(1000):
    pattern.simulate_pattern(backend="statevector", input_state=states)
    if pattern.results[4]==0 and pattern.results[5]==0:
        n_good_output += 1

print(f"Number of occurrences of output '00' : {n_good_output}")



N, node = 2
N, node = 3
N, node = 4
N, node = 5
E, nodes = (1, 2)
E, nodes = (2, 0)
E, nodes = (2, 3)
E, nodes = (0, 4)
E, nodes = (3, 5)
M, node = 1, plane = Plane.XY, angle(pi) = 0.0, s_domain = set(), t_domain = set()
M, node = 2, plane = Plane.XY, angle(pi) = 0.0, s_domain = set(), t_domain = set()
M, node = 0, plane = Plane.XY, angle(pi) = 0.0, s_domain = set(), t_domain = {1}
M, node = 3, plane = Plane.XY, angle(pi) = 0.0, s_domain = {2}, t_domain = {1}
X byproduct, node = 4, domain = {0}
X byproduct, node = 5, domain = {3}
Z byproduct, node = 5, domain = {2}
Number of occurrences of output '00' : 1000


In [3]:


secrets = Secrets(r=True, a=True, theta=True)

client = Client(pattern=pattern, secrets=secrets, input_state=states)

test_runs = client.create_test_runs()


number_of_traps = sum([len(run.trap_qubits)
                    for run in test_runs])
n_nodes = len(client.graph[0])
print(f"The graph of the pattern has {n_nodes} nodes.")
print(f"There are {number_of_traps} traps in total. (VBQC uses single-qubit traps)")

The graph of the pattern has 6 nodes.
There are 6 traps in total. (VBQC uses single-qubit traps)


## Concrete VBQC example

### Performing the simulation

In [4]:
import random as rd

from graphix.sim.density_matrix import DensityMatrixBackend

from noise_model import VBQCNoiseModel

In [5]:
from matplotlib import cm  # Colormap
from matplotlib.colors import LogNorm

# Trappified scheme parameters
d = 50 # nr of computation rounds
t = 50 # nr of test rounds
N = d + t
rounds = list(range(N))
rd.shuffle(rounds)

# Re-setting the backend
backend = DensityMatrixBackend()

# Store data for each value of p
all_histograms = {}
failed_traps_histograms = {}

# Example of multiple p values (noise levels)
p_values = [0, 0.1, 0.15, 0.2, 0.5, 1]

print(f"Iterating {N} times with {d} computation rounds and {t} trap rounds.")
for p in p_values:

    # Defining the noise model (depolarizing noise)
    noise=VBQCNoiseModel(
        entanglement_error_prob=p
    )
    print(f"Attempt with noise model p={p}")

    # Recording outcomes/traps failures
    outcomes_histogram = dict()
    n_failed_traps = 0

    # Iterating through rounds
    for i in rounds:
        if i < d:
            # Computation round
            client.refresh_randomness()
            client.delegate_pattern(backend=backend, noise_model=noise)

            # Store result (increment occurrence in histogram)
            computation_outcome = ""
            for onode in classical_output:
                computation_outcome += str(int(client.results[onode]))
            if computation_outcome not in outcomes_histogram:
                outcomes_histogram[computation_outcome] = 1
            else:
                outcomes_histogram[computation_outcome] += 1
        else:
            # Test round
            run = rd.choice(test_runs)
            client.refresh_randomness()
            trap_outcomes = client.delegate_test_run(run=run, backend=backend, noise_model=noise)

            # Record trap failure
            # A trap round fails if one of the single-qubit traps failed
            if sum(trap_outcomes) != 0:
                n_failed_traps += 1

    # Combine results
    all_histograms[p] = outcomes_histogram
    if t!= 0:
        failed_traps_histograms[p]=n_failed_traps/(t)

# Combine all unique outcomes from all histograms
all_outcomes = sorted(set().union(*[hist.keys() for hist in all_histograms.values()]))
print("Proportion of failed traps in for different noise levels: ", failed_traps_histograms)


Iterating 100 times with 50 computation rounds and 50 trap rounds.
Attempt with noise model p=0
Attempt with noise model p=0.1
Attempt with noise model p=0.15
Attempt with noise model p=0.2
Attempt with noise model p=0.5
Attempt with noise model p=1
Proportion of failed traps in for different noise levels:  {0: 0.0, 0.1: 0.28, 0.15: 0.34, 0.2: 0.58, 0.5: 0.78, 1: 0.84}


### Plotting the results (plotly)

### Histogram for the computation rounds

In [6]:
import plotly.graph_objects as go
from matplotlib.colors import to_hex


# Function to get color based on p value
def get_heatmap_color(p, colormap_name='coolwarm', vmin=0.1, vmax=1):
    colormap = cm.get_cmap(colormap_name)
    norm = LogNorm(vmin=vmin, vmax=vmax)  # Normalize p within [vmin, vmax]
    return to_hex(colormap(norm(p)))  # Convert to hex color for Plotly

# Initialize the figure
fig = go.Figure()

# Store the p values in a list to access them by index in buttons
p_values = list(all_histograms.keys())

# Add histograms for different 'p' values
for p, histogram in all_histograms.items():
    frequencies = [histogram.get(outcome, 0) for outcome in all_outcomes]

    # Get colors for each bar based on the current p value
    colors = [get_heatmap_color(p + 0.1) for _ in all_outcomes]

    fig.add_trace(
        go.Bar(
            x=all_outcomes,
            y=frequencies,
            name=f'p={p}',  # Legend label
            marker=dict(color=colors),  # Apply custom colors
            visible=(p == 0)  # Only p=0 is visible by default
        )
    )

fig.update_layout(
    title="Output bitstring occurrences, per noise level p",
    updatemenus=[
        {
            'type': 'buttons',
            'direction': 'right',
            'x': 0.5,  # Horizontal position of the buttons
            'y': 1.15,  # Vertical position above the chart
            'xanchor': 'center',  # Anchor buttons horizontally center
            'buttons': [
                {
                    'label': f'p= {key}',
                    'method': 'update',
                    'args': [{'visible': [i == idx for i in range(len(all_histograms))]}]
                }
                for idx, key in enumerate(all_histograms)
            ] + [
                {
                    'label': 'Show All',
                    'method': 'update',
                    'args': [{'visible': [True] * len(all_histograms)}]
                }
            ]
        }
    ]
)

# Set initial visibility (show only the first chart by default)
initial_visibility = [True] + [False] * (len(all_histograms) - 1)
fig.update_traces(visible=initial_visibility[0])

fig.show()


/var/folders/n5/hv7xz3zn675bw8ckw4q5ll3cmp3czq/T/ipykernel_28784/908283072.py:7: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



## Verification: do we accept these outcomes ?
### Displaying the proportion of failed traps

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Function to get color based on p value
def get_heatmap_color(p, colormap_name='coolwarm', vmin=0.1, vmax=1):
    colormap = cm.get_cmap(colormap_name)
    norm = LogNorm(vmin=vmin, vmax=vmax)  # Normalize p within [vmin, vmax]
    return to_hex(colormap(norm(p)))  # Convert to hex color for Plotly

# Initialize the figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.7, 0.3],
    subplot_titles=("Output Bitstring Occurrences", "Failed/Passed Traps"),
    specs=[[{"type": "bar"}, {"type": "pie"}]]
)

# Store the p values in a list to access them by index in buttons
p_values = list(all_histograms.keys())

# Add bar charts and pie charts for different 'p' values
colors = ['#EF553B', '#a5ff33']  # Red for 'Failed traps', Green for 'Passed traps'

for _idx, (p, histogram) in enumerate(all_histograms.items()):
    frequencies = [histogram.get(outcome, 0) for outcome in all_outcomes]

    # Get colors for each bar based on the current p value
    bar_colors = [get_heatmap_color(p + 0.1) for _ in all_outcomes]

    # Bar chart
    fig.add_trace(
        go.Bar(
            x=all_outcomes,
            y=frequencies,
            name=f'p={p}',  # Legend label
            marker=dict(color=bar_colors),  # Apply custom colors
            visible=(p == p_values[0])  # Only the first p is visible by default
        ),
        row=1, col=1
    )

    # Pie chart
    failed_value = failed_traps_histograms.get(p, 0)  # Fetch failed traps for current p
    fig.add_trace(
        go.Pie(
            labels=['Failed traps', 'Passed traps'],
            values=[failed_value, 1 - failed_value],
            marker=dict(colors=colors),
            name=f'p={p}',
            visible=(p == p_values[0])
        ),
        row=1, col=2
    )

# Update layout with buttons
fig.update_layout(
    title="Output bitstring occurrences and trap results per noise level p",
    updatemenus=[
        {
            'type': 'buttons',
            'direction': 'right',
            'x': 0.5,
            'y': 1.15,
            'xanchor': 'center',
            'buttons': [
                {
                    'label': f'p= {key}',
                    'method': 'update',
                    'args': [{'visible': [(i // 2 == idx) for i in range(2 * len(all_histograms))]}]
                }
                for idx, key in enumerate(all_histograms)
            ] + [
                {
                    'label': 'Show All',
                    'method': 'update',
                    'args': [{'visible': [True] * (2 * len(all_histograms))}]
                }
            ]
        }
    ]
)

fig.show()


/var/folders/n5/hv7xz3zn675bw8ckw4q5ll3cmp3czq/T/ipykernel_28784/4244744428.py:8: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Function to get color based on p value
def get_heatmap_color(p, colormap_name='coolwarm', vmin=0.1, vmax=1):
    colormap = cm.get_cmap(colormap_name)
    norm = LogNorm(vmin=vmin, vmax=vmax)  # Normalize p within [vmin, vmax]
    return to_hex(colormap(norm(p)))  # Convert to hex color for Plotly

# Number of rows and columns for the subplot grid
num_rows = len(all_histograms)  # Each p value gets its own row
num_cols = 2  # One column for bar chart, one for pie chart

# Create a subplot grid with multiple rows and columns
fig = make_subplots(
    rows=num_rows, cols=num_cols,
    column_widths=[0.7, 0.3],
    row_titles=[f'p={p}' for p in all_histograms.keys()],
    subplot_titles=["Output Bitstring Occurrences", "Failed/Passed Traps"] * len(all_histograms),
    vertical_spacing=0.05,
    specs=[[{"type": "bar"}, {"type": "pie"}] for _ in range(num_rows)]
)

# Add bar charts and pie charts for each p value
colors = ['#EF553B', '#a5ff33']  # Red for 'Failed traps', Green for 'Passed traps'

for idx, (p, histogram) in enumerate(all_histograms.items()):
    frequencies = [histogram.get(outcome, 0) for outcome in all_outcomes]

    # Get colors for each bar based on the current p value
    bar_colors = [get_heatmap_color(p + 0.1) for _ in all_outcomes]

    # Add bar chart to the first column
    fig.add_trace(
        go.Bar(
            x=all_outcomes,
            y=frequencies,
            name=f'Bitstrings (p={p})',
            marker=dict(color=bar_colors)
        ),
        row=idx + 1, col=1
    )

    # Add pie chart to the second column
    failed_value = failed_traps_histograms.get(p, 0)
    fig.add_trace(
        go.Pie(
            labels=['Failed traps', 'Passed traps'],
            values=[failed_value, 1 - failed_value],
            marker=dict(colors=colors),
            name=f'Traps (p={p})'
        ),
        row=idx + 1, col=2
    )

# Update layout for better spacing and appearance
fig.update_layout(
    title="Output Bitstring Occurrences and Trap Results per Noise Level (p)",
    height=400 * num_rows,  # Dynamic height based on the number of rows
    showlegend=False  # Disable legend for clarity
)

fig.show()


/var/folders/n5/hv7xz3zn675bw8ckw4q5ll3cmp3czq/T/ipykernel_28784/2457274934.py:8: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.

